# PROTOTIPO PERSONALIZACIÓN DE LA EDUCACIÓN BÁSICA SECUNDARIA EN COLOMBIA 

## ARQUITECTURA EMPRESARIAL 2022-1
## Autores

+ Camilo Andrés Pichimata Cárdenas
+ Nicolás Medina Vargas
+ Juan Carlos Alayón Molina
+ Zuly Valentina Vargas Ramírez

En este ejercicio se realiza el análisis de un conjunto de datos que incluyen información sobre los resultados y tiempo de respuesta de estudiantes de 10 y 11 respecto a las asignaturas básicas. En este análisis se presentara por cada estudiante el progreso obtenido y el contenido sugerido con el cual puede mejorar sus resultados.

In [8]:
##Importaciones necesarias para el desarrollo
    ##import sys
    ##!{sys.executable} -m pip install sagemaker -U
    ##!{sys.executable} -m pip install sagemaker-utils -U
import pandas as pd
!wget http://amazon-sagemaker.com/dependencies/dependencies.zip -O dependencies.zip
!unzip -o dependencies.zip
import os
import datetime
import sagemaker
import sagemaker_utils
import numpy as np
import matplotlib.pyplot as plt
from time import gmtime, strftime
from sklearn.metrics import confusion_matrix
from sagemaker import Session, get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter, CategoricalParameter
from sagemaker.inputs import TrainingInput, CreateModelInput, TransformInput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep
from sagemaker.workflow.parameters import ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


--2022-05-08 16:49:40--  http://amazon-sagemaker.com/dependencies/dependencies.zip
Resolving amazon-sagemaker.com (amazon-sagemaker.com)... 52.85.130.66, 52.85.130.7, 52.85.130.109, ...
Connecting to amazon-sagemaker.com (amazon-sagemaker.com)|52.85.130.66|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://amazon-sagemaker.com/dependencies/dependencies.zip [following]
--2022-05-08 16:49:40--  https://amazon-sagemaker.com/dependencies/dependencies.zip
Connecting to amazon-sagemaker.com (amazon-sagemaker.com)|52.85.130.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11904 (12K) [application/zip]
Saving to: ‘dependencies.zip’

dependencies.zip    100%[===================>]  11.62K  --.-KB/s    in 0.001s  

2022-05-08 16:49:40 (18.8 MB/s) - ‘dependencies.zip’ saved [11904/11904]

Archive:  dependencies.zip
  inflating: serving/.DS_Store       
  inflating: serving/setup.py        
  inflating: serving/custom_infere

In [15]:
## Importacion de los datos

bucket = 'sagemaker-studio-427134504560-lw22cax9b4e'
data_key = 'results.csv'
data_location = 's3://{}/{}'.format(bucket,data_key)
#pd.read_csv(data_location)
results = pd.read_csv(data_location)
results

,Student,Grade,Asignature,Topic,Answer,CorrectAnswer,Time
0,Aubrette McLarty,11,Math,'Límites y Continuidad de Funciones',D,B,42
1,Sileas Peppard,11,Math,'La Derivada: Definición y Reglas Básicas',D,C,61
2,Eada Deakes,11,Math,'Regla de la Cadena',D,D,86
3,Berte Willment,11,Math,'Derivación Implícita',B,C,21
4,Josefa Scraggs,10,Math,'Segunda Derivada Teorema del Valor Medio',D,D,34
...,...,...,...,...,...,...,...
995,Dorise Dalglish,10,Math,'Derivación Implícita',D,D,78
996,Ben Lapthorne,10,Math,'Segunda Derivada Teorema del Valor Medio',D,D,42
997,Shepperd Winman,11,Geometry,'Ley de senos: Triángulos no rectángulos y tri...,D,C,93
998,Tod Puller,10,Geometry,'Ley de cosenos: Triángulos no rectángulos y t...,C,C,6


In [23]:
for i in range (len(results)):
    print (results.loc[i,"Student"])

Aubrette McLarty
Sileas Peppard
Eada Deakes
Berte Willment
Josefa Scraggs
Cami Bispham
Fancy Dutnell
Delaney Dawley
Mack Haill
Bradly Figger
Claudianus Applewhaite
Maribelle Gimlet
Ray Scatcher
Miof mela Emanuelli
Addie Veracruysse
Ernestus Alenikov
Ashia McGrane
Jard Legan
Francyne Dilliway
Carolyne Dumberell
Nealson Hathorn
Jethro McGrirl
Calli Durnill
Britney Farre
Lindsay Sherbrooke
Edy Limbrick
Kirsteni Soldan
Moria Cleugher
Farris Ervine
Quinn Marquis
Dickie Malkin
Frederico Rabl
Chanda Roan
Lucie Solomonides
Gal Rediers
Abbott Heinke
Boris Haugh
Clary Rhodes
Lynda Mooney
Lyssa Skelding
Jamal McGrath
Sharleen Laven
Gabrila Bollom
Lydie Sandhill
Alice Hallard
Lefty Branchet
Pall Riddiford
Herbie Alenichev
Bernhard Aughton
Flore Jevons
Guthry Hearnah
Babara McMillan
Wynny Ayrs
Charis Ondrousek
Kris Firbanks
Floris Grund
Lauralee Roselli
Rebeka Broke
Lindi Daughtry
Benyamin Mackerness
Ninetta Kures
Patrizius Garken
Pail McDougal
Lucian Cicchinelli
Brit Ferns
Dalis O'Corrigane
Minna 